## 과제 1
ReLu activation function과 derivative function을 구현해보세요
- Hint : np.maximum 함수 사용하면 편리합니다
- 다른 방법 사용하셔도 무방합니다


In [1]:
import numpy as np

In [2]:
def relu(x):

  return np.maximum(0, x)

In [3]:
def d_relu(x):
  if x < 0:
    return 0
  return 1

## 과제 2
Deep Learning Basic 코드 파일의 MLP implementation with Numpy library using MNIST dataset 코드 참고해서
Three layer MLP 일 때의 backward_pass 함수를 완성해주세요.   
- Hint : 코드 파일의 예시는 Two layer MLP


In [4]:
def backward_pass(x, y_true, params):

  dS3 = params["A3"] - y_true

  grads = {}

  grads["dW3"] = np.dot(dS3, params["A2"].T)/x.shape[1]
  grads["db3"] = (1/x.shape[1])*np.sum(dS3, axis=1, keepdims=True)/x.shape[1]

  dA2 = np.dot(params["W3"].T, dS3)
  dS2 = dA2 * d_sigmoid(params["S2"])

  grads["dW2"] =  np.dot(dS2, params["A1"].T)/x.shape[1]
  grads["db2"] =  (1/x.shape[1])*np.sum(dS2, axis=1, keepdims=True)/x.shape[1]

  dA1 = np.dot(params["W2"].T, dS2)
  dS1 = dA1 * d_sigmoid(params["S1"])

  grads["dW1"] = np.dot(dS1, x.T)/x.shape[1]
  grads["db1"] = np.sum(dS1, axis=1, keepdims=True)/x.shape[1]

  return grads

## 과제 3
Deep Learning Basic 코드 파일의 MLP implementation with Pytorch library using MNIST dataset 코드 참고해서
Three layer MLP를 구한후, 학습을 돌려 보세요

hyperparameter는 다음과 같이 설정

- epochs : 100
- hiddensize : 128, 64 (two layer)
- learning_rate : 0.5

In [ ]:
# Assignment 3 구현은 여기서 ()

In [8]:
from torchvision import transforms, datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [9]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [10]:
trainset = datasets.MNIST(
    root      = './.data/', 
    train     = True,
    download  = True,
    transform = transform
)
testset = datasets.MNIST(
    root      = './.data/', 
    train     = False,
    download  = True,
    transform = transform
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/train-images-idx3-ubyte.gz to ./.data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/train-labels-idx1-ubyte.gz to ./.data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/MNIST/raw



In [11]:
trainset[0][0].shape

torch.Size([1, 28, 28])

In [12]:
train_loader = DataLoader(trainset, batch_size=32, shuffle=True)
test_loader =  DataLoader(testset, batch_size=32, shuffle=False)

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(784,128)
        self.layer2 = nn.Linear(128,64)
        self.layer3 = nn.Linear(64,10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 784)
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        out = self.relu(out)
        out = self.layer3(out)

        return out

In [14]:
model = Net()
model

Net(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.5)

In [18]:
def train(model, train_loader, optimizer):
    model.train()
    # 배치 당 loss 값을 담을 리스트 생성
    batch_losses = []

    for data, target in train_loader:
        # 옵티마이저의 기울기 초기화
        optimizer.zero_grad()

        # y pred 값 산출
        output = model(data)
        # loss 계산
        # 정답 데이터와의 cross entropy loss 계산
        # 이 loss를 배치 당 loss로 보관
        loss = criterion(output, target)
        batch_losses.append(loss)

        # 기울기 계산
        loss.backward()

        # 가중치 업데이트!
        optimizer.step()
        
    # 배치당 평균 loss 계산
    avg_loss = sum(batch_losses) / len(batch_losses)
    
    return avg_loss

In [19]:
def evaluate(model, test_loader):
    # 모델을 평가 모드로 전환
    model.eval()

    batch_losses = []
    correct = 0 

    with torch.no_grad(): 
        for data, target in test_loader:
            # 예측값 생성
            output = model(data)

            # loss 계산 (이전과 동일)
            loss = criterion(output, target)
            batch_losses.append(loss)

           # Accuracy 계산
           # y pred와 y가 일치하면 correct에 1을 더해주기
            pred = output.max(1, keepdim=True)[1]

            # eq() 함수는 값이 일치하면 1을, 아니면 0을 출력.
            correct += pred.eq(target.view_as(pred)).sum().item()

    # 배치 당 평균 loss 계산 
    avg_loss =  sum(batch_losses) / len(batch_losses)

    #정확도 계산
    accuracy = 100. * correct / len(test_loader.dataset)

    return avg_loss, accuracy

In [20]:
EPOCHS = 100

for epoch in range(1, EPOCHS + 1):
    train_loss = train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Train Loss: {:.4f}\tTest Loss: {:.4f}\tAccuracy: {:.2f}%'.format(
          epoch, train_loss, test_loss, test_accuracy))

[1] Train Loss: 0.2949	Test Loss: 0.2140	Accuracy: 93.36%
[2] Train Loss: 0.1249	Test Loss: 0.1403	Accuracy: 95.87%
[3] Train Loss: 0.0936	Test Loss: 0.0994	Accuracy: 97.19%
[4] Train Loss: 0.0778	Test Loss: 0.0979	Accuracy: 97.18%
[5] Train Loss: 0.0651	Test Loss: 0.1147	Accuracy: 96.87%
[6] Train Loss: 0.0601	Test Loss: 0.1154	Accuracy: 97.30%
[7] Train Loss: 0.0540	Test Loss: 0.1041	Accuracy: 97.47%
[8] Train Loss: 0.0549	Test Loss: 0.1298	Accuracy: 97.02%
[9] Train Loss: 0.0437	Test Loss: 0.1373	Accuracy: 96.88%
[10] Train Loss: 0.0384	Test Loss: 0.1501	Accuracy: 96.54%
[11] Train Loss: 0.0467	Test Loss: 0.1125	Accuracy: 97.51%
[12] Train Loss: 0.0410	Test Loss: 0.1194	Accuracy: 97.24%
[13] Train Loss: 0.0381	Test Loss: 0.1179	Accuracy: 97.26%
[14] Train Loss: 0.0362	Test Loss: 0.1769	Accuracy: 96.66%
[15] Train Loss: 0.0326	Test Loss: 0.1393	Accuracy: 97.43%
[16] Train Loss: 0.0401	Test Loss: 0.1464	Accuracy: 97.24%
[17] Train Loss: 0.0353	Test Loss: 0.1329	Accuracy: 97.70%
[18] T

## 과제 4
과제 3 부분의 성능을 지금까지 배운 지식을 바탕으로 향상시켜보세요

- Hint : Activation function, hyperparameter setting

In [ ]:
# Assignment 4 구현은 여기서 ()

In [21]:
model2 = Net()
model2

Net(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)

In [22]:
optimizer2 = optim.SGD(model2.parameters(), lr=0.01)

In [23]:
EPOCHS = 50

for epoch in range(1, EPOCHS + 1):
    train_loss = train(model2, train_loader, optimizer2)
    test_loss, test_accuracy = evaluate(model2, test_loader)
    
    print('[{}] Train Loss: {:.4f}\tTest Loss: {:.4f}\tAccuracy: {:.2f}%'.format(
          epoch, train_loss, test_loss, test_accuracy))

[1] Train Loss: 1.1868	Test Loss: 0.4345	Accuracy: 87.68%
[2] Train Loss: 0.3775	Test Loss: 0.3137	Accuracy: 90.83%
[3] Train Loss: 0.3026	Test Loss: 0.2669	Accuracy: 92.19%
[4] Train Loss: 0.2611	Test Loss: 0.2324	Accuracy: 93.20%
[5] Train Loss: 0.2296	Test Loss: 0.2109	Accuracy: 94.04%
[6] Train Loss: 0.2049	Test Loss: 0.1907	Accuracy: 94.45%
[7] Train Loss: 0.1851	Test Loss: 0.1747	Accuracy: 94.95%
[8] Train Loss: 0.1685	Test Loss: 0.1628	Accuracy: 95.18%
[9] Train Loss: 0.1553	Test Loss: 0.1521	Accuracy: 95.63%
[10] Train Loss: 0.1437	Test Loss: 0.1418	Accuracy: 95.84%
[11] Train Loss: 0.1330	Test Loss: 0.1360	Accuracy: 95.91%
[12] Train Loss: 0.1238	Test Loss: 0.1280	Accuracy: 96.26%
[13] Train Loss: 0.1150	Test Loss: 0.1194	Accuracy: 96.50%
[14] Train Loss: 0.1078	Test Loss: 0.1152	Accuracy: 96.46%
[15] Train Loss: 0.1012	Test Loss: 0.1105	Accuracy: 96.68%
[16] Train Loss: 0.0953	Test Loss: 0.1057	Accuracy: 96.88%
[17] Train Loss: 0.0897	Test Loss: 0.1029	Accuracy: 96.93%
[18] T

**무엇을 보완하였고, 왜 보완되었는지에 대한 자유 서술 (아래에)**

In [ ]:
lr이 0.5로 너무 커서 더 작은 값으로 조정했다.

In [ ]:
epoch이 너무 커서 줄였다.